In [23]:
import os
import pandas as pd
from pandas import read_csv, read_excel, DataFrame
from sae import COTSCar
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum as qsum
import numpy as np
from typing import List

import sys
sys.path.append('../')
import saedfsc

Price of the car.

In [24]:
productPrice = 20000
maxProductPrice = 25000
pricePerf = (maxProductPrice - productPrice) / maxProductPrice # Normalized price (0 is best, 1 is worst)

Information about customers

In [25]:
customers = saedfsc.customers['Name'].to_list()
cQty = dict(zip(saedfsc.customers['Name'], saedfsc.customers['Quantity'])) # customer quantities
cPriceFocus = dict(zip(saedfsc.customers['Name'], saedfsc.customers['PriceFocus']))
name_weights_dict = saedfsc.customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWts = {c : np.fromstring(name_weights_dict[c].strip('[]'), sep=',') for c in name_weights_dict}

Create a COTSCar object and compute the market share for each customer type (based on logit model).

In [30]:
def getTotalUtilityForCustomer(car : COTSCar, c : str):
    total_utility = 0
    perf_utility = car.partworth_objectives(weights=cWts[c])[0]
    total_utility += (1-cPriceFocus[c])*perf_utility - cPriceFocus[c]*pricePerf
    return total_utility

def getMarketShare(car : COTSCar, c : str, competitors : List[COTSCar]): # based on logit model of demand
    carUtility = getTotalUtilityForCustomer(car, c)
    totalCompetitorUtility = sum([getTotalUtilityForCustomer(car, c) for car in competitors])
    return carUtility / (totalCompetitorUtility + carUtility)

competitors = []
for i in range(3):
    competitorCar = COTSCar()
    competitors.append(competitorCar)
    print("Competitor Car " + str(i) + " has design: ", competitorCar.vector)

carDesignVec = [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
product = COTSCar()
product.vector = carDesignVec
print("Our Car has design: ", product.vector)

market_shares = {c : getMarketShare(product, c, competitors) for c in customers}
expected_customer_demand = {c : getMarketShare(product, c, competitors)*cQty[c] for c in customers}
market_shares_df = pd.DataFrame(list(market_shares.items()), columns=['Customer Type', 'MarketShare'])
customer_demand_df = pd.DataFrame(list(expected_customer_demand.items()), columns=['Customer Type', 'Expected Demand'])
df = pd.merge(market_shares_df, customer_demand_df, on='Customer Type')
df

Competitor Car 0 has design:  [3, 12, 2, 4, 3, 0, 1, 0, 5, 1, 35, 116, 3, 3, 111, 194, 191]
Competitor Car 1 has design:  [9, 10, 8, 10, 2, 0, 1, 11, 12, 3, 56, 133, 0, 2, 207, 140, 66]
Competitor Car 2 has design:  [9, 6, 4, 2, 0, 2, 4, 6, 12, 1, 37, 108, 1, 2, 140, 198, 80]
Our Car has design:  [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]


,Customer Type,MarketShare,Expected Demand
0,CustomerType1,0.228289,186.968477
1,CustomerType2,0.285447,81.637756
2,CustomerType3,0.155209,74.034689
3,CustomerType4,0.170561,123.145024
4,CustomerType5,0.103875,33.551520
5,CustomerType6,0.138910,81.817863
6,CustomerType7,0.121613,101.789815
7,CustomerType8,0.083289,16.074818
8,CustomerType9,0.195890,17.630102
9,CustomerType10,0.133343,70.138229
